# Ever Given

<img align="right" src="https://movingpandas.github.io/movingpandas/assets/img/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/movingpandas/movingpandas-examples/main?filepath=2-analysis-examples/ever-given.ipynb)
[![IPYNB](https://img.shields.io/badge/view-ipynb-hotpink)](https://github.com/movingpandas/movingpandas-examples/blob/main/2-analysis-examples/ever-given.ipynb)
[![HTML](https://img.shields.io/badge/view-html-green)](https://movingpandas.github.io/movingpandas-website/2-analysis-examples/ever-given.html)

This notebook presents an analysis of the vessel situation following the grounding of Ever Given in the Suez Canal.


The dataset used covers the time span between 2021-03-20 00:00 and 2021-03-24 12:52 UTC.
 
This data has generously been provided by VesselsValue.


<img align="left" src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/Container_Ship_%27Ever_Given%27_stuck_in_the_Suez_Canal%2C_Egypt_-_March_24th%2C_2021_cropped.jpg/320px-Container_Ship_%27Ever_Given%27_stuck_in_the_Suez_Canal%2C_Egypt_-_March_24th%2C_2021_cropped.jpg">


In [ ]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
from shapely.geometry import Point
from datetime import datetime, timedelta
from hvplot import pandas

from holoviews import opts, dim
from holoviews.selection import link_selections

import warnings

warnings.simplefilter("ignore")

In [ ]:
EVERID = 235
FSIZE = 300

In [ ]:
df = pd.read_csv("../data/boat-positions.csv")
df["t"] = pd.to_datetime(df["ais_pos_timestamp"], format="%d/%m/%Y %H:%M")
df

In [ ]:
gdf = gpd.GeoDataFrame(
    df.drop(["longitude", "latitude", "ais_pos_timestamp"], axis=1),
    crs="epsg:4326",
    geometry=[Point(xy) for xy in zip(df.longitude, df.latitude)],
)
gdf

In [ ]:
gdf.hvplot(geo=True, tiles="OSM", frame_width=FSIZE, frame_height=FSIZE)

In [ ]:
tc = mpd.TrajectoryCollection(gdf, "ID", t="t")

In [ ]:
evergiven = tc.get_trajectory(EVERID)
evergiven.hvplot(line_width=3, frame_width=FSIZE, frame_height=FSIZE)

In [ ]:
stop_detector = mpd.TrajectoryStopDetector(tc)
stop_pts = stop_detector.get_stop_points(
    min_duration=timedelta(hours=3), max_diameter=1000
)
stop_pts

In [ ]:
stop_pts["duration_h"] = stop_pts["duration_s"] / 3600
stop_pts

## Ever Given ran aground around 7:40 local time (5:40 UTC) on 23rd March 2021

In [ ]:
stop_pts[stop_pts["traj_id"] == EVERID]

In [ ]:
map_plot = (
    stop_pts.hvplot(geo=True, hover_cols=["start_time"], size=20, tiles="OSM")
    * evergiven.hvplot(
        line_width=5,
        color="red",
        frame_width=FSIZE,
        frame_height=FSIZE,
        alpha=0.5,
        tiles=None,
    ).opts(active_tools=["pan", "wheelzoom"])
    * stop_pts[stop_pts["traj_id"] == EVERID].hvplot(
        geo=True,
        hover_cols=["start_time"],
        size=dim("duration_h") / 2,
        color="red",
        title="Trajectory & stop location of Ever Given and stops of other vessels",
    )
)
scatter_plot = stop_pts.hvplot.scatter(
    title="Stop start & duration (in hours)",
    x="start_time",
    y="duration_h",
    frame_width=FSIZE,
    frame_height=FSIZE,
    alpha=0.7,
) * stop_pts[stop_pts["traj_id"] == EVERID].hvplot.scatter(
    x="start_time", y="duration_h", color="red", size=200
)
map_plot + scatter_plot

*Data generously provided by VesselsValue.*

In [ ]:
stop_pts[stop_pts.start_time > datetime(2021, 3, 23, 5, 39, 0)].sort_values(
    "duration_s", ascending=False
).head(
    12
)  # .style.background_gradient(cmap='Reds')

## Continue exploring MovingPandas

1. [Bird migration analysis](bird-migration.ipynb)
1. [Ship data analysis](ship-data.ipynb)
1. [Horse collar data exploration](horse-collar.ipynb)
1. [OSM traces](osm-traces.ipynb)
1. [Soccer game](soccer-game.ipynb)
1. [Mars rover & heli](mars-rover.ipynb)
1. [Ever Given](ever-given.ipynb)
1. [Iceberg](iceberg.ipynb)
1. [Pollution data](pollution-data.ipynb)